In [11]:
import boto3
import json
from botocore.exceptions import ClientError
import logging
logger = logging.getLogger(__name__)

In [2]:
client = boto3.client(service_name='secretsmanager')

In [16]:
def get_secret():

    secret_name = "test/MySQLProxy"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e
    else:
        secret = get_secret_value_response['SecretString']
        return json.loads(secret)

In [17]:
secret = get_secret()
print(secret['username'])

admin


In [18]:
secret

{'username': 'admin',
 'password': 'passworD$',
 'engine': 'mysql',
 'host': 'database-1.cluster-cywjvduoxnim.us-east-1.rds.amazonaws.com',
 'port': 3306,
 'dbClusterIdentifier': 'database-1'}

{'username': 'admin',
 'password': 'passworD$',
 'engine': 'mysql',
 'host': 'database-1.cluster-cywjvduoxnim.us-east-1.rds.amazonaws.com',
 'port': 3306,
 'dbClusterIdentifier': 'database-1'}

In [58]:
def create_secret(secret_name, secret_string):
    try:
        response = client.create_secret(
            Name=secret_name,
            SecretString=secret_string
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceExistsException':
            logger.error("The secret already exists: %s", secret_name)
            raise
        else:
            logger.error("Error: %s", e)
            raise
    else:
        print("Secret created successfully.")

In [60]:
name="DatabaseProdFun"
sec_string='{"username": "prod", "password": "hello-world"}'

create_secret(name, sec_string)

Secret created successfully.


True

In [13]:
def get_secret(secret_name):
    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print("The requested secret " + secret_name + " was not found")
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            print("The request was invalid due to:", e)
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            print("The request had invalid params:", e)
        elif e.response['Error']['Code'] == 'DecryptionFailure':
            print("The requested secret can't be decrypted using the provided KMS key:", e)
        elif e.response['Error']['Code'] == 'InternalServiceError':
            print("An error occurred on service side:", e)
    else:
        # Secrets Manager decrypts the secret value using the associated KMS CMK
        # Depending on whether the secret was a string or binary, only one of these fields will be populated
        if 'SecretString' in get_secret_value_response:
            return get_secret_value_response['SecretString']
        else:
            return get_secret_value_response['SecretBinary']

In [34]:
sect = get_secret("DatabaseProdFun")

In [35]:
sect

'{"username": "prod", "password": "hello-world"}'